In [3]:
import pandas as pd
import numpy as np
import ta 

In [4]:
dataSet = pd.read_csv('EURUSD_M30-1.csv')

In [5]:
dataSet = dataSet.sort_values(by ='Time', ascending=False)
dataSet = dataSet.dropna()
time_col = dataSet['Time']
dataSet.columns = dataSet.columns.str.strip().str.capitalize()
dataSet['Time'] = time_col

In [6]:
dataSet['rsi'] = ta.momentum.RSIIndicator(close=dataSet['Close']).rsi()
dataSet['ema_20'] = ta.trend.EMAIndicator(dataSet['Close'], window=20).ema_indicator()
dataSet['atr'] = ta.volatility.AverageTrueRange(dataSet['High'], dataSet['Low'], dataSet['Close']).average_true_range()


In [12]:
for i in range(1, 4):
    dataSet[f'return_lag_{i}'] = dataSet['Close'].pct_change(i)
# dataSet['hour'] = dataSet['Time'].dt.hour
# dataSet['day_of_week'] = dataSet.index.dayofweek

# dataSet.dropna(inplace=True)
dataSet.dtypes

Time             object
Open            float64
High            float64
Low             float64
Close           float64
Volume            int64
rsi             float64
ema_20          float64
atr             float64
return_lag_1    float64
return_lag_2    float64
return_lag_3    float64
dtype: object

In [ ]:
dataSet.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'rsi', 'ema_20', 'atr'], dtype='object')